### read sentiment data and merge with post information

In [1]:
import glob
import pandas as pd

sentiments_api = pd.DataFrame({
    '0':[],
    '1':[],
    '2':[],
    '3':[],
    '4':[],
    '5':[]
})

for n in range(len(glob.glob('./Reddit_post_api/*.csv'))):
    df = pd.read_csv(glob.glob('./Reddit_post_api/*.csv')[n])
    sentiments_api = pd.concat( [sentiments_api, df], axis=0)
    
print(sentiments_api.shape)
sentiments_api.columns = ['post_id','anger','joy','sadness','fear','disgust']

(389494, 6)


In [2]:
post = pd.read_csv('processed_reddit_post_vadder.csv')
print(post.shape)
#print(post[:6])

(389494, 9)


In [3]:
merged_post = pd.merge(post, sentiments_api, how='left', on=['post_id'])

In [5]:
del merged_post['text']
del merged_post['num_comments']
del merged_post['sentiment_vader']
del merged_post['score']

### find posts before 2018-04-24 01:30:00 

In [20]:
all_timestamp[30:40]

['2016-04-21 19:00:00',
 '2016-04-24 22:00:00',
 '2016-04-22 04:00:00',
 '2016-04-23 04:00:00',
 '2016-04-25 04:00:00',
 '2016-04-08 07:00:00',
 '2016-04-14 11:00:00',
 '2016-04-15 00:00:00',
 '2016-04-18 20:00:00',
 '2016-04-25 01:00:00']

In [21]:
all_timestamp = merged_post['timestamp'].values.tolist()
train_timestamp_index = []
for i in range(len(all_timestamp)):
    if int(all_timestamp[i][:4]) < 2018:
        train_timestamp_index.append(i)
    elif (all_timestamp[i][:4] == '2018') and (int(all_timestamp[i][6]) < 4):
        train_timestamp_index.append(i)
    elif (all_timestamp[i][:7] == '2018-04') and (int(all_timestamp[i][8:10]) < 24):
        train_timestamp_index.append(i)
    elif (all_timestamp[i][:13] == '2018-04-24 00') or (all_timestamp[i][:13] == '2018-04-24 01'):
        train_timestamp_index.append(i)

In [24]:
len(train_timestamp_index)

329079

In [25]:
train_merged_post = merged_post.loc[train_timestamp_index,:]

In [26]:
train_merged_post.shape

(329079, 10)

### add a column to notify which sentiment has highest value
#### 'Positive' represents for joy; 'Negative' represents for anger, sadness, fear and disgust

In [27]:
train_merged_post['max_sentiment'] = '0'

In [38]:
train_merged_post.loc[:][5:10]

,Unnamed: 0,post_id,post_author,post_score,timestamp,anger,joy,sadness,fear,disgust,max_sentiment
5,5,4dmftl,mkabatek,143,2016-04-07 02:00:00,0.017638,0.013868,0.017937,0.019820,0.017834,0
6,6,4gue65,Galwoa,194,2016-04-29 01:00:00,0.022014,0.011442,0.018262,0.023088,0.022474,0
7,7,4gjthm,Coinosphere,74,2016-04-27 03:00:00,0.022473,0.016751,0.009394,0.015313,0.013343,0
8,8,4foatp,mmeijeri,352,2016-04-21 02:00:00,0.006465,0.024178,0.009585,0.004310,0.003999,0
9,9,4fa3qf,collegeworker,25,2016-04-18 13:00:00,0.021436,0.009034,0.019839,0.030982,0.024871,0


In [47]:
%%time
for i in train_merged_post.index.values.tolist():
    sentiments = train_merged_post.loc[i][5:10].values.tolist()
    #print(max(sentiments))
    max_senti_location = sentiments.index(max(sentiments))
    #print(max_senti_location)
    if max_senti_location == 1:
        train_merged_post.loc[i,'max_sentiment'] = 'Positive'
    else:
        train_merged_post.loc[i,'max_sentiment'] = 'Negative'

CPU times: user 54min 43s, sys: 28.1 s, total: 55min 11s
Wall time: 1h 1s


In [48]:
train_merged_post[:2]

,Unnamed: 0,post_id,post_author,post_score,timestamp,anger,joy,sadness,fear,disgust,max_sentiment
0,0,4dvm9x,mrpg_,70,2016-04-08 19:00:00,0.004330,0.025898,0.003477,0.006926,0.004544,Positive
1,1,4ezxyu,BitScrip,22,2016-04-16 11:00:00,0.017532,0.015844,0.012073,0.018675,0.015834,Negative


### find the users with higher score

In [49]:
score = train_merged_post['post_score'].values.tolist()
score.sort(reverse=True)
print(score[-20:])
#threshold_score = score[int(len(score)/3*2)]
threshold_score = train_merged_post['post_score'].values.mean()
print(threshold_score)

score = train_merged_post['post_score'].values.tolist()
high_score = [0 for i in range(len(score))]

index_over_threshold = []
for i in range(len(score)):
    if score[i] > threshold_score:
        high_score[i] = 1
        index_over_threshold.append(i)
    else:
        high_score[i] = 0
        
train_merged_post['high_score'] = high_score
#print(index_over_threshold)
threshold_score = 28

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
27.867733887607535


### compare sentiment with actual price changing

In [50]:
%%time
data_BTUSD = pd.read_csv('/Users/lizhe/Desktop/CMU/ISproject/gemini_BTCUSD_1hr.csv')

headers = ['Unix Timestamp','Date','Symbol','Open','High','Low','Close','Volume']
data_BTUSD = pd.read_csv('/Users/lizhe/Desktop/CMU/ISproject/gemini_BTCUSD_1hr.csv',names=headers)
data_BTUSD=data_BTUSD.drop([0,1,2])

# set same form of the time in the dataframes
data_BTUSD['Date']=pd.to_datetime(data_BTUSD['Date'])
train_merged_post['timestamp']=pd.to_datetime(train_merged_post['timestamp'])
train_merged_post['timestamp']=pd.Series(train_merged_post['timestamp']).dt.round("H")


price = data_BTUSD['Close'].values.tolist()

BT_timestamp = data_BTUSD['Date'].values.tolist()
post_timestamp = train_merged_post['timestamp'].values.tolist()

# find the actual average bitcoin price change
price_actual_trend = []
count_valueError = 0
for i in index_over_threshold:
    sum_price_end = 0
    try:
        index_begin = BT_timestamp.index(post_timestamp[i])
        price_begin = float(price[index_begin])
        for j in range(1,52):
            index_end = index_begin + j
            sum_price_end += float(price[index_end])
        price_end = sum_price_end/51
        if price_end >= price_begin:
            price_actual_trend.append('Positive')
        else:
            price_actual_trend.append('Negative')
    except ValueError as e:
        count_valueError += 1
        price_actual_trend.append('Unknown')
        
print('NotAvailable:', price_actual_trend.count('NotAvailable'))
print(len(price_actual_trend))

merged_post_high_score = train_merged_post.loc[index_over_threshold,:]
#print(merged_post_high_score[:3])
merged_post_high_score['price_actual_trend'] = price_actual_trend
merged_post_high_score[:2]

NotAvailable: 0
33341
CPU times: user 11.4 s, sys: 149 ms, total: 11.5 s
Wall time: 11.9 s


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [51]:
post_timestamp = train_merged_post['timestamp'].values.tolist()
print(post_timestamp[:5])
print(BT_timestamp[:5])
print(data_BTUSD['Date'][:3])
print(train_merged_post['timestamp'][:3])
print(len(index_over_threshold))

[1460142000000000000, 1460804400000000000, 1461549600000000000, 1461092400000000000, 1460217600000000000]
[1558915200000000000, 1558911600000000000, 1558908000000000000, 1558904400000000000, 1558900800000000000]
3   2019-05-27 00:00:00
4   2019-05-26 23:00:00
5   2019-05-26 22:00:00
Name: Date, dtype: datetime64[ns]
0   2016-04-08 19:00:00
1   2016-04-16 11:00:00
2   2016-04-25 02:00:00
Name: timestamp, dtype: datetime64[ns]
33341


In [60]:
accurate_prediction = []
for i in merged_post_high_score.index.values.tolist():
    if merged_post_high_score.loc[i][10] == merged_post_high_score.loc[i][12]:
        accurate_prediction.append(1)
    else:
        accurate_prediction.append(0)

merged_post_high_score['accurate_prediction'] = accurate_prediction


In [15]:
# print(sum(merged_post_high_score['accurate_prediction'].values.tolist()))
# print(merged_post_high_score.loc[1][14], merged_post_high_score.loc[1][16])

### Calculate the prediction accuracy for users with high scores 

In [64]:
users = []
accuracy = []
for i in merged_post_high_score.index.tolist():
    name = merged_post_high_score.loc[i]['post_author']
    if users.count(name) == 0:
        users.append(name)

In [65]:
print(len(users),len(merged_post_high_score.index.tolist()))

15141 33341


In [66]:
%%time
all_users = merged_post_high_score['post_author'].values.tolist()
merged_post_high_score.index = range(len(all_users))
accuracy = []

users_predict_once = []
predict_more_than_once = []
users_60_higher_accuracy = []
users_70_higher_accuracy = []
users_80_higher_accuracy = []


for i in range(len(users)):
    user_id = users[i]
    total_pred_times = all_users.count(user_id)
    if total_pred_times == 1:
        #index = all_users.index(user_id)
        accurate_pred_times = 0 # merged_post_high_score['accurate_prediction'][index]
        users_predict_once.append(user_id)
    else:
        all_pred = [0 for i in range(total_pred_times)]
        index = [0 for i in range(total_pred_times)]
        for i in range(total_pred_times):
            index[i] = all_users.index(user_id)
            all_users[index[i]] = 0
            all_pred[i] = merged_post_high_score['accurate_prediction'][index[i]]
        accurate_pred_times = sum(all_pred)
        predict_more_than_once.append(user_id)
        if accurate_pred_times/total_pred_times >= 0.8:
            users_60_higher_accuracy.append(user_id)
            users_70_higher_accuracy.append(user_id)
            users_80_higher_accuracy.append(user_id)
        elif accurate_pred_times/total_pred_times >= 0.7:
            users_60_higher_accuracy.append(user_id)
            users_70_higher_accuracy.append(user_id)
        elif accurate_pred_times/total_pred_times >= 0.6:
            users_60_higher_accuracy.append(user_id)
    pred_accuracy_this_user = accurate_pred_times / total_pred_times
    accuracy.append(pred_accuracy_this_user)
    
print(accuracy[:10])

[0.0, 0.5454545454545454, 0.4, 0.0, 0.5, 0.4, 0.46153846153846156, 0.0, 0.0, 0.0]
CPU times: user 17.7 s, sys: 68.6 ms, total: 17.8 s
Wall time: 18.3 s


In [67]:
count_60_accuracy = 0
count_70_accuracy = 0
count_80_accuracy = 0
for i in accuracy:
    if i >= 0.8:
        count_60_accuracy += 1
        count_70_accuracy += 1
        count_80_accuracy += 1
    elif i >= 0.7:
        count_70_accuracy += 1
        count_60_accuracy += 1
    elif i >= 0.6:
        count_60_accuracy += 1
        
print('count_60_accuracy:',count_60_accuracy, '; % of all high-score users',count_60_accuracy/len(users)*100,'%')
print('count_70_accuracy:',count_70_accuracy, '; % of all high-score users',count_70_accuracy/len(users)*100,'%')
print('count_80_accuracy:',count_80_accuracy, '; % of all high-score users',count_80_accuracy/len(users)*100,'%')
print(len(users))

count_60_accuracy: 1174 ; % of all high-score users 7.753781124100126 %
count_70_accuracy: 788 ; % of all high-score users 5.204411861832112 %
count_80_accuracy: 660 ; % of all high-score users 4.359025163463444 %
15141


### Add weights to the post whose author has high prediction accuracy

In [68]:
train_merged_post[:3]

,Unnamed: 0,post_id,post_author,post_score,timestamp,anger,joy,sadness,fear,disgust,max_sentiment,high_score
0,0,4dvm9x,mrpg_,70,2016-04-08 19:00:00,0.004330,0.025898,0.003477,0.006926,0.004544,Positive,1
1,1,4ezxyu,BitScrip,22,2016-04-16 11:00:00,0.017532,0.015844,0.012073,0.018675,0.015834,Negative,0
2,2,4g8ssg,gonzobon,5,2016-04-25 02:00:00,0.012624,0.015986,0.017008,0.017521,0.016553,Negative,0


#### Output the influencer names and their weight

In [69]:
%%time
#### all_users = merged_post_high_score['post_author'].values.tolist()
score = merged_post_high_score['post_score'].values.tolist()

weight_users_60_higher_accuracy = []

conversion_list_weight_calculation = [0 for i in range(len(users_60_higher_accuracy))]
conversion_list_users = []
conversion_list_pred_times = [0 for i in range(len(users_60_higher_accuracy))]

for i in range(len(all_users)):
    post_author = merged_post_high_score['post_author'][i]
    if post_author in users_60_higher_accuracy:
        index = users_60_higher_accuracy.index(post_author)
        if post_author not in conversion_list_users:
            conversion_list_users.append(post_author)
            conversion_list_weight_calculation[index] = score[i]
            conversion_list_pred_times[index] = 1
        else:
            conversion_list_weight_calculation[index] += score[i]
            conversion_list_pred_times[index] += 1

for i in range(len(users_60_higher_accuracy)):
    weight0 = (conversion_list_weight_calculation[i]/conversion_list_pred_times[i])/threshold_score
    weight_users_60_higher_accuracy.append(weight0)
    
print(users_60_higher_accuracy[:5])
print(weight_users_60_higher_accuracy[:5])

['trymad', 'jameslwalpole', 'kraken_suspended', 'stickac', 'bit79']
[2.25, 4.613636363636364, 6.214285714285714, 3.988095238095238, 3.5714285714285716]
CPU times: user 1.14 s, sys: 5.9 ms, total: 1.14 s
Wall time: 1.17 s


In [70]:
user_and_weight = pd.DataFrame({
    'post_author':users_60_higher_accuracy,
    'weight':weight_users_60_higher_accuracy
})
user_and_weight.to_csv('./reddit_influencer_and_weight_new.csv', sep=',', header=True)

#### Output all users and their weighted sentiments 

In [77]:
%%time
all_users_any_score = merged_post['post_author'].values.tolist()
avg_score = train_merged_post.loc[:,'post_score'].mean()
weight = []
for i in range(len(all_users_any_score)):
    author = merged_post['post_author'][i]
    if author in users_60_higher_accuracy:
        index = users_60_higher_accuracy.index(author)
        weight_found = weight_users_60_higher_accuracy[index]
        weight.append(weight_found)
    else:
        weight.append(1)

CPU times: user 11.2 s, sys: 28.9 ms, total: 11.2 s
Wall time: 11.3 s


In [78]:
print(len(weight))
merged_post.shape

389494


(389494, 10)

In [79]:
merged_post['weight'] = weight

In [80]:
weighted_anger = []
weighted_joy = []
weighted_sadness = []
weighted_fear = []
weighted_disgust = []

for i in range(len(all_users_any_score)):
    weighted_anger.append(float(merged_post['anger'][i])*weight[i])
    weighted_joy.append(float(merged_post['joy'][i])*weight[i])
    weighted_sadness.append(float(merged_post['sadness'][i])*weight[i])
    weighted_fear.append(float(merged_post['fear'][i])*weight[i])
    weighted_disgust.append(float(merged_post['disgust'][i])*weight[i])
    
merged_post['weighted_anger'] = weighted_anger
merged_post['weighted_joy'] = weighted_joy
merged_post['weighted_sadness'] = weighted_sadness
merged_post['weighted_fear'] = weighted_fear
merged_post['weighted_disgust'] = weighted_disgust

In [81]:
merged_post[40:43]

,Unnamed: 0,post_id,post_author,post_score,timestamp,anger,joy,sadness,fear,disgust,weight,weighted_anger,weighted_joy,weighted_sadness,weighted_fear,weighted_disgust
40,40,4dah6e,vampireban,19,2016-04-04 20:00:00,0.013824,0.018217,0.012952,0.013612,0.012844,1.000000,0.013824,0.018217,0.012952,0.013612,0.012844
41,41,4edmdi,goodbtc,42,2016-04-12 10:00:00,0.017660,0.017136,0.009323,0.017655,0.014996,1.664286,0.029391,0.028519,0.015516,0.029383,0.024957
42,42,4ctiqf,TLDReynolds,287,2016-04-01 13:00:00,0.022347,0.011083,0.018321,0.024176,0.021911,1.000000,0.022347,0.011083,0.018321,0.024176,0.021911


In [84]:
# del merged_post['text']
# del merged_post['num_comments']
# del merged_post['sentiment_vader']


In [85]:
merged_post[40:43]

,Unnamed: 0,post_id,post_author,post_score,timestamp,anger,joy,sadness,fear,disgust,weight,weighted_anger,weighted_joy,weighted_sadness,weighted_fear,weighted_disgust
40,40,4dah6e,vampireban,19,2016-04-04 20:00:00,0.013824,0.018217,0.012952,0.013612,0.012844,1.000000,0.013824,0.018217,0.012952,0.013612,0.012844
41,41,4edmdi,goodbtc,42,2016-04-12 10:00:00,0.017660,0.017136,0.009323,0.017655,0.014996,1.664286,0.029391,0.028519,0.015516,0.029383,0.024957
42,42,4ctiqf,TLDReynolds,287,2016-04-01 13:00:00,0.022347,0.011083,0.018321,0.024176,0.021911,1.000000,0.022347,0.011083,0.018321,0.024176,0.021911


In [86]:
merged_post.to_csv('./reddit_influencer_with_weighted_sentiment_new.csv', sep=',', header=True, index=False)

In [87]:
merged_post.shape

(389494, 16)